# Monte Carlo Methods in Reinforcement Learning
## Applied to Blackjack Environment

This notebook demonstrates Monte Carlo methods for reinforcement learning using the Blackjack environment from Gymnasium.

### What we'll cover:
1. Monte Carlo Evaluation (Prediction)
2. Monte Carlo Control
3. Why we estimate Q-values instead of V-values
4. Transforming MDP to MRP
5. Constant-α Monte Carlo implementation

In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import random
from tqdm import tqdm
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

print("Libraries imported successfully!")

## Monte Carlo Methods Theory

### What are Monte Carlo Methods?
Monte Carlo methods learn value functions and optimal policies from **complete episodes** of experience. Unlike dynamic programming, they don't require knowledge of the environment's dynamics.

### Key Characteristics:
- **Model-free**: No need to know transition probabilities
- **Episode-based**: Learn from complete episodes
- **Sample-based**: Use actual returns rather than expected returns

### Monte Carlo Evaluation (Prediction)
Goal: Estimate the value function V^π(s) for a given policy π

**Algorithm:**
1. Generate episodes following policy π
2. For each state visited, calculate the return G_t
3. Average returns to estimate V(s)

### Monte Carlo Control
Goal: Find the optimal policy π*

**Algorithm:**
1. Policy Evaluation: Estimate Q^π(s,a)
2. Policy Improvement: Update policy greedily
3. Repeat until convergence

## Why Estimate Q-values instead of V-values?

### The Problem with V-values in Model-free Settings

In model-free reinforcement learning, we don't know the environment dynamics P(s',r|s,a). 

**Policy Improvement with V-values requires:**
```
π(s) = argmax_a Σ_{s',r} P(s',r|s,a)[r + γV(s')]
```

**Problem:** We don't know P(s',r|s,a)!

### Solution: Use Q-values

**Policy Improvement with Q-values:**
```
π(s) = argmax_a Q(s,a)
```

**Advantage:** No need for environment model!

### MDP to MRP Transformation

When we fix a policy π, we transform the **Markov Decision Process (MDP)** into a **Markov Reward Process (MRP)**:

**MDP:** (S, A, P, R, γ)
**MRP:** (S, P^π, R^π, γ)

Where:
- P^π(s'|s) = Σ_a π(a|s)P(s'|s,a)
- R^π(s) = Σ_a π(a|s)R(s,a)

This transformation allows us to evaluate the value function for a fixed policy.

In [ ]:
# Create Blackjack environment
env = gym.make('Blackjack-v1', sab=True)  # sab=True for state-action-based rewards

print("Blackjack Environment:")
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")
print("\nState representation: (player_sum, dealer_card, usable_ace)")
print("Actions: 0 = Stick, 1 = Hit")

# Test environment
state, info = env.reset()
print(f"\nSample initial state: {state}")
print(f"Player sum: {state[0]}, Dealer card: {state[1]}, Usable ace: {state[2]}")

In [ ]:
def generate_episode(env, policy):
    """
    Generate a complete episode following the given policy
    Returns: list of (state, action, reward) tuples
    """
    episode = []
    state, _ = env.reset()
    
    while True:
        action = policy(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        episode.append((state, action, reward))
        
        if terminated or truncated:
            break
        state = next_state
    
    return episode

def simple_policy(state):
    """
    Simple policy: Hit if player sum < 20, else stick
    """
    player_sum = state[0]
    return 1 if player_sum < 20 else 0

def epsilon_greedy_policy(Q, state, epsilon=0.1):
    """
    Epsilon-greedy policy for exploration
    """
    if random.random() < epsilon:
        return random.choice([0, 1])  # Random action
    else:
        return 0 if Q[state][0] > Q[state][1] else 1  # Greedy action

print("Helper functions defined!")

In [ ]:
def monte_carlo_evaluation(env, policy, num_episodes=10000, gamma=1.0):
    """
    Monte Carlo evaluation to estimate V^π(s)
    """
    # Initialize
    V = defaultdict(float)
    returns = defaultdict(list)
    
    print(f"Running Monte Carlo Evaluation for {num_episodes} episodes...")
    
    for episode_num in tqdm(range(num_episodes)):
        # Generate episode
        episode = generate_episode(env, policy)
        
        # Calculate returns for each state
        G = 0
        visited_states = set()
        
        # Work backwards through episode
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            
            # First-visit MC
            if state not in visited_states:
                returns[state].append(G)
                V[state] = np.mean(returns[state])
                visited_states.add(state)
    
    return dict(V)

# Run Monte Carlo Evaluation
V_simple = monte_carlo_evaluation(env, simple_policy, num_episodes=50000)
print(f"\nEstimated {len(V_simple)} state values")
print("Sample state values:")
for i, (state, value) in enumerate(list(V_simple.items())[:5]):
    print(f"State {state}: V = {value:.3f}")

In [ ]:
def constant_alpha_monte_carlo_control(env, num_episodes=100000, alpha=0.01, epsilon=0.1, gamma=1.0):
    """
    Constant-α Monte Carlo Control
    Uses incremental updates instead of averaging all returns
    """
    # Initialize Q-values
    Q = defaultdict(lambda: np.zeros(2))  # 2 actions: stick(0), hit(1)
    
    print(f"Running Constant-α Monte Carlo Control...")
    print(f"Episodes: {num_episodes}, α: {alpha}, ε: {epsilon}")
    
    episode_rewards = []
    
    for episode_num in tqdm(range(num_episodes)):
        # Generate episode using ε-greedy policy
        episode = []
        state, _ = env.reset()
        episode_reward = 0
        
        # Generate episode
        while True:
            action = epsilon_greedy_policy(Q, state, epsilon)
            next_state, reward, terminated, truncated, _ = env.step(action)
            episode.append((state, action, reward))
            episode_reward += reward
            
            if terminated or truncated:
                break
            state = next_state
        
        episode_rewards.append(episode_reward)
        
        # Update Q-values using constant-α
        G = 0
        visited_pairs = set()
        
        # Work backwards through episode
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            
            # First-visit update
            if (state, action) not in visited_pairs:
                # Constant-α update: Q(s,a) ← Q(s,a) + α[G - Q(s,a)]
                Q[state][action] += alpha * (G - Q[state][action])
                visited_pairs.add((state, action))
    
    return dict(Q), episode_rewards

# Run Constant-α Monte Carlo Control
Q_optimal, rewards = constant_alpha_monte_carlo_control(
    env, 
    num_episodes=100000, 
    alpha=0.01, 
    epsilon=0.1
)

print(f"\nLearned Q-values for {len(Q_optimal)} states")
print("Training completed!")

In [ ]:
def extract_policy(Q):
    """
    Extract greedy policy from Q-values
    """
    policy = {}
    for state in Q:
        policy[state] = np.argmax(Q[state])
    return policy

def policy_function(state, policy_dict):
    """
    Policy function that can be used with generate_episode
    """
    if state in policy_dict:
        return policy_dict[state]
    else:
        # Default action for unseen states
        return 0  # Stick

# Extract optimal policy
optimal_policy_dict = extract_policy(Q_optimal)
print(f"Optimal policy extracted for {len(optimal_policy_dict)} states")

# Create policy function
def optimal_policy(state):
    return policy_function(state, optimal_policy_dict)

print("Optimal policy function created!")

In [ ]:
def visualize_policy_interactive(policy_dict, title="Policy Visualization"):
    """
    Interactive visualization of the policy
    """
    print(f"\n{title}")
    print("=" * len(title))
    
    # Organize states by usable ace
    no_ace_states = {}
    ace_states = {}
    
    for state, action in policy_dict.items():
        player_sum, dealer_card, usable_ace = state
        if usable_ace:
            ace_states[(player_sum, dealer_card)] = action
        else:
            no_ace_states[(player_sum, dealer_card)] = action
    
    def print_policy_table(states_dict, title):
        print(f"\n{title}:")
        print("Player\\Dealer", end="")
        for dealer in range(1, 11):
            print(f"{dealer:>4}", end="")
        print()
        
        for player in range(12, 22):
            print(f"{player:>11}", end="")
            for dealer in range(1, 11):
                if (player, dealer) in states_dict:
                    action = states_dict[(player, dealer)]
                    symbol = "H" if action == 1 else "S"
                    print(f"{symbol:>4}", end="")
                else:
                    print(f"-{4}", end="")
            print()
    
    print_policy_table(no_ace_states, "No Usable Ace")
    print_policy_table(ace_states, "Usable Ace")
    
    print("\nLegend: H = Hit, S = Stick")

# Visualize the optimal policy
visualize_policy_interactive(optimal_policy_dict, "Optimal Policy from Monte Carlo Control")

In [ ]:
def test_policy(env, policy, num_episodes=10000):
    """
    Test a policy and return average reward
    """
    total_reward = 0
    wins = 0
    losses = 0
    draws = 0
    
    for _ in range(num_episodes):
        episode = generate_episode(env, policy)
        episode_reward = sum([reward for _, _, reward in episode])
        total_reward += episode_reward
        
        if episode_reward > 0:
            wins += 1
        elif episode_reward < 0:
            losses += 1
        else:
            draws += 1
    
    avg_reward = total_reward / num_episodes
    win_rate = wins / num_episodes
    
    return {
        'avg_reward': avg_reward,
        'win_rate': win_rate,
        'wins': wins,
        'losses': losses,
        'draws': draws
    }

# Test different policies
print("Testing Policies...")
print("=" * 50)

# Test simple policy
simple_results = test_policy(env, simple_policy, 10000)
print("\nSimple Policy (Hit if sum < 20):")
print(f"Average Reward: {simple_results['avg_reward']:.4f}")
print(f"Win Rate: {simple_results['win_rate']:.4f}")
print(f"Wins: {simple_results['wins']}, Losses: {simple_results['losses']}, Draws: {simple_results['draws']}")

# Test optimal policy
optimal_results = test_policy(env, optimal_policy, 10000)
print("\nOptimal Policy (Monte Carlo):")
print(f"Average Reward: {optimal_results['avg_reward']:.4f}")
print(f"Win Rate: {optimal_results['win_rate']:.4f}")
print(f"Wins: {optimal_results['wins']}, Losses: {optimal_results['losses']}, Draws: {optimal_results['draws']}")

improvement = optimal_results['avg_reward'] - simple_results['avg_reward']
print(f"\nImprovement: {improvement:.4f} ({improvement/abs(simple_results['avg_reward'])*100:.1f}%)")

In [ ]:
# Plot learning progress
def plot_learning_progress(rewards, window_size=1000):
    """
    Plot the learning progress over episodes
    """
    # Calculate moving average
    moving_avg = []
    for i in range(len(rewards)):
        start_idx = max(0, i - window_size + 1)
        moving_avg.append(np.mean(rewards[start_idx:i+1]))
    
    plt.figure(figsize=(12, 6))
    
    # Plot raw rewards (sampled for visibility)
    sample_indices = np.arange(0, len(rewards), max(1, len(rewards)//1000))
    plt.subplot(1, 2, 1)
    plt.plot(sample_indices, [rewards[i] for i in sample_indices], alpha=0.3, color='lightblue')
    plt.plot(moving_avg, color='red', linewidth=2, label=f'Moving Average (window={window_size})')
    plt.xlabel('Episode')
    plt.ylabel('Episode Reward')
    plt.title('Learning Progress: Episode Rewards')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot moving average only
    plt.subplot(1, 2, 2)
    plt.plot(moving_avg, color='red', linewidth=2)
    plt.xlabel('Episode')
    plt.ylabel('Average Reward')
    plt.title('Learning Progress: Moving Average')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Final average reward (last {window_size} episodes): {np.mean(rewards[-window_size:]):.4f}")

# Plot the learning progress
plot_learning_progress(rewards, window_size=5000)

In [ ]:
def interactive_policy_explorer(Q_values):
    """
    Interactive exploration of the learned policy
    """
    print("\n" + "="*60)
    print("INTERACTIVE POLICY EXPLORER")
    print("="*60)
    
    def explore_state(player_sum, dealer_card, usable_ace):
        state = (player_sum, dealer_card, usable_ace)
        
        if state in Q_values:
            q_stick = Q_values[state][0]
            q_hit = Q_values[state][1]
            optimal_action = "STICK" if q_stick > q_hit else "HIT"
            
            print(f"\nState: Player={player_sum}, Dealer={dealer_card}, Ace={'Yes' if usable_ace else 'No'}")
            print(f"Q(s, STICK) = {q_stick:.4f}")
            print(f"Q(s, HIT)   = {q_hit:.4f}")
            print(f"Optimal Action: {optimal_action}")
            print(f"Advantage: {abs(q_stick - q_hit):.4f}")
        else:
            print(f"\nState not visited during training: Player={player_sum}, Dealer={dealer_card}, Ace={'Yes' if usable_ace else 'No'}")
    
    # Example explorations
    print("\nExploring some interesting states:")
    
    interesting_states = [
        (20, 10, False),  # Strong hand vs strong dealer
        (16, 10, False),  # Weak hand vs strong dealer
        (12, 6, False),   # Weak hand vs weak dealer
        (18, 9, True),    # Soft 18 vs 9
        (13, 2, True),    # Soft 13 vs 2
        (11, 10, False),  # 11 vs 10
    ]
    
    for state in interesting_states:
        explore_state(*state)
    
    return explore_state

# Run interactive explorer
explore_function = interactive_policy_explorer(Q_optimal)

## Monte Carlo Method Analysis

### Key Insights from Our Implementation:

#### 1. **Constant-α vs Sample Average**
- **Sample Average**: Q(s,a) = average of all returns
- **Constant-α**: Q(s,a) ← Q(s,a) + α[G - Q(s,a)]
- **Advantage of Constant-α**: Adapts to non-stationary environments, gives more weight to recent experience

#### 2. **Why Q-values Work Better**
- Direct policy improvement without environment model
- π(s) = argmax_a Q(s,a) is straightforward
- No need for transition probabilities

#### 3. **MDP → MRP Transformation**
When we fix policy π:
- **Original MDP**: Multiple actions per state
- **Resulting MRP**: Single action per state (determined by π)
- **Benefit**: Can use MRP solution methods for evaluation

#### 4. **Episode-based Learning**
- Must wait for episode completion
- Good for episodic tasks like Blackjack
- Natural handling of terminal states

#### 5. **Exploration vs Exploitation**
- ε-greedy ensures all state-action pairs are visited
- Critical for finding optimal policy
- Balance between learning and performance

In [ ]:
def every_visit_monte_carlo_control(env, num_episodes=50000, alpha=0.01, epsilon=0.1, gamma=1.0):
    """
    Every-visit Monte Carlo Control (vs First-visit)
    Updates Q-values for every occurrence of (s,a) in an episode
    """
    Q = defaultdict(lambda: np.zeros(2))
    
    print(f"Running Every-Visit Monte Carlo Control...")
    
    for episode_num in tqdm(range(num_episodes)):
        # Generate episode
        episode = []
        state, _ = env.reset()
        
        while True:
            action = epsilon_greedy_policy(Q, state, epsilon)
            next_state, reward, terminated, truncated, _ = env.step(action)
            episode.append((state, action, reward))
            
            if terminated or truncated:
                break
            state = next_state
        
        # Update Q-values for every visit (not just first visit)
        G = 0
        
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            
            # Every-visit update (no check for previous visits)
            Q[state][action] += alpha * (G - Q[state][action])
    
    return dict(Q)

def off_policy_monte_carlo_control(env, num_episodes=50000, alpha=0.01, gamma=1.0):
    """
    Off-policy Monte Carlo Control using Importance Sampling
    Target policy: greedy, Behavior policy: ε-greedy
    """
    Q = defaultdict(lambda: np.zeros(2))
    C = defaultdict(lambda: np.zeros(2))  # Cumulative weights
    
    epsilon = 0.1  # For behavior policy
    
    print(f"Running Off-Policy Monte Carlo Control...")
    
    for episode_num in tqdm(range(num_episodes)):
        # Generate episode using behavior policy (ε-greedy)
        episode = []
        state, _ = env.reset()
        
        while True:
            action = epsilon_greedy_policy(Q, state, epsilon)
            next_state, reward, terminated, truncated, _ = env.step(action)
            episode.append((state, action, reward))
            
            if terminated or truncated:
                break
            state = next_state
        
        # Importance sampling update
        G = 0
        W = 1  # Importance sampling ratio
        
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            
            # Update with importance sampling
            C[state][action] += W
            Q[state][action] += (W / C[state][action]) * (G - Q[state][action])
            
            # Check if action matches target policy (greedy)
            greedy_action = np.argmax(Q[state])
            if action != greedy_action:
                break  # Trajectory ends for target policy
            
            # Update importance sampling ratio
            behavior_prob = epsilon/2 + (1-epsilon) * (action == greedy_action)
            target_prob = 1.0  # Greedy policy probability
            W *= target_prob / behavior_prob
    
    return dict(Q)

print("Advanced Monte Carlo methods defined!")

In [ ]:
# Compare different Monte Carlo variants
print("Comparing Monte Carlo Variants...")
print("=" * 50)

# Run every-visit Monte Carlo
print("\n1. Every-Visit Monte Carlo:")
Q_every_visit = every_visit_monte_carlo_control(env, num_episodes=30000)
every_visit_policy = extract_policy(Q_every_visit)

def every_visit_policy_func(state):
    return policy_function(state, every_visit_policy)

every_visit_results = test_policy(env, every_visit_policy_func, 5000)
print(f"Average Reward: {every_visit_results['avg_reward']:.4f}")
print(f"Win Rate: {every_visit_results['win_rate']:.4f}")

# Run off-policy Monte Carlo
print("\n2. Off-Policy Monte Carlo:")
Q_off_policy = off_policy_monte_carlo_control(env, num_episodes=30000)
off_policy_policy = extract_policy(Q_off_policy)

def off_policy_policy_func(state):
    return policy_function(state, off_policy_policy)

off_policy_results = test_policy(env, off_policy_policy_func, 5000)
print(f"Average Reward: {off_policy_results['avg_reward']:.4f}")
print(f"Win Rate: {off_policy_results['win_rate']:.4f}")

# Compare all methods
print("\n" + "=" * 60)
print("COMPARISON OF ALL METHODS")
print("=" * 60)
print(f"{'Method':<25} {'Avg Reward':<12} {'Win Rate':<10}")
print("-" * 50)
print(f"{'Simple Policy':<25} {simple_results['avg_reward']:<12.4f} {simple_results['win_rate']:<10.4f}")
print(f"{'First-Visit MC':<25} {optimal_results['avg_reward']:<12.4f} {optimal_results['win_rate']:<10.4f}")
print(f"{'Every-Visit MC':<25} {every_visit_results['avg_reward']:<12.4f} {every_visit_results['win_rate']:<10.4f}")
print(f"{'Off-Policy MC':<25} {off_policy_results['avg_reward']:<12.4f} {off_policy_results['win_rate']:<10.4f}")

In [ ]:
def step_by_step_monte_carlo_demo():
    """
    Detailed step-by-step demonstration of Monte Carlo learning
    """
    print("\n" + "="*70)
    print("STEP-BY-STEP MONTE CARLO DEMONSTRATION")
    print("="*70)
    
    # Initialize
    Q = defaultdict(lambda: np.zeros(2))
    alpha = 0.1
    epsilon = 0.1
    
    print(f"\nInitial Setup:")
    print(f"Learning rate (α): {alpha}")
    print(f"Exploration rate (ε): {epsilon}")
    print(f"Q-values initialized to 0")
    
    # Demonstrate a few episodes
    for episode_num in range(3):
        print(f"\n{'='*50}")
        print(f"EPISODE {episode_num + 1}")
        print(f"{'='*50}")
        
        # Generate episode
        episode = []
        state, _ = env.reset()
        print(f"\nInitial state: {state}")
        
        step = 0
        while True:
            # Choose action
            if random.random() < epsilon:
                action = random.choice([0, 1])
                action_type = "Random (exploration)"
            else:
                action = np.argmax(Q[state])
                action_type = "Greedy (exploitation)"
            
            print(f"\nStep {step + 1}:")
            print(f"  State: {state}")
            print(f"  Q(s,STICK): {Q[state][0]:.3f}, Q(s,HIT): {Q[state][1]:.3f}")
            print(f"  Action: {'HIT' if action == 1 else 'STICK'} ({action_type})")
            
            # Take action
            next_state, reward, terminated, truncated, _ = env.step(action)
            episode.append((state, action, reward))
            
            print(f"  Reward: {reward}")
            
            if terminated or truncated:
                print(f"  Episode terminated")
                break
            
            state = next_state
            step += 1
        
        # Update Q-values
        print(f"\nUpdating Q-values:")
        G = 0
        visited_pairs = set()
        
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = G + reward  # gamma = 1 for simplicity
            
            if (state, action) not in visited_pairs:
                old_q = Q[state][action]
                Q[state][action] += alpha * (G - Q[state][action])
                new_q = Q[state][action]
                
                print(f"  State {state}, Action {'HIT' if action == 1 else 'STICK'}:")
                print(f"    Return G = {G}")
                print(f"    Old Q = {old_q:.3f}")
                print(f"    New Q = {old_q:.3f} + {alpha} * ({G} - {old_q:.3f}) = {new_q:.3f}")
                
                visited_pairs.add((state, action))
        
        print(f"\nEpisode {episode_num + 1} complete!")
    
    print(f"\n{'='*70}")
    print("DEMONSTRATION COMPLETE")
    print(f"{'='*70}")
    print("\nKey observations:")
    print("1. Each episode provides complete return information")
    print("2. Q-values are updated using actual returns (not estimates)")
    print("3. Exploration ensures we visit different state-action pairs")
    print("4. Learning happens after each complete episode")

# Run the demonstration
step_by_step_monte_carlo_demo()

## Summary and Key Takeaways

### What We've Accomplished:

1. **Implemented Monte Carlo Evaluation**
   - Estimated value functions from episode samples
   - Used first-visit averaging method
   - Demonstrated model-free learning

2. **Implemented Monte Carlo Control**
   - Found optimal policy through Q-value estimation
   - Used ε-greedy exploration strategy
   - Applied constant-α updates for adaptability

3. **Explored Advanced Techniques**
   - Every-visit vs First-visit methods
   - Off-policy learning with importance sampling
   - Comparative analysis of different approaches

### Key Insights:

#### Why Q-values over V-values?
- **Model-free policy improvement**: π(s) = argmax_a Q(s,a)
- **No environment dynamics needed**: Direct action selection
- **Practical implementation**: Works in unknown environments

#### MDP to MRP Transformation:
- **Fixed policy creates MRP**: Single action per state
- **Evaluation becomes simpler**: Standard MRP solution methods
- **Theoretical foundation**: Justifies Monte Carlo evaluation

#### Constant-α Benefits:
- **Adaptive learning**: Recent experience weighted more
- **Non-stationary environments**: Handles changing dynamics
- **Faster convergence**: In many practical scenarios

### Monte Carlo Advantages:
- ✅ Model-free learning
- ✅ Unbiased value estimates
- ✅ Simple implementation
- ✅ Natural episode handling

### Monte Carlo Limitations:
- ❌ Requires complete episodes
- ❌ High variance estimates
- ❌ Slow convergence
- ❌ Only for episodic tasks

### Next Steps:
- **Temporal Difference Learning**: Combines MC and DP benefits
- **Function Approximation**: Handle large state spaces
- **Policy Gradient Methods**: Direct policy optimization

In [ ]:
def interactive_exercises():
    """
    Interactive exercises for deeper understanding
    """
    print("\n" + "="*60)
    print("INTERACTIVE EXERCISES")
    print("="*60)
    
    print("\nExercise 1: Policy Comparison")
    print("-" * 30)
    
    def conservative_policy(state):
        """Very conservative policy - stick early"""
        player_sum = state[0]
        return 1 if player_sum < 17 else 0
    
    def aggressive_policy(state):
        """Aggressive policy - hit more often"""
        player_sum = state[0]
        return 1 if player_sum < 19 else 0
    
    # Test policies
    conservative_results = test_policy(env, conservative_policy, 5000)
    aggressive_results = test_policy(env, aggressive_policy, 5000)
    
    print(f"Conservative Policy (stick at 17+): {conservative_results['avg_reward']:.4f}")
    print(f"Aggressive Policy (stick at 19+):   {aggressive_results['avg_reward']:.4f}")
    print(f"Optimal MC Policy:                   {optimal_results['avg_reward']:.4f}")
    
    print("\nExercise 2: Learning Rate Impact")
    print("-" * 30)
    
    alphas = [0.001, 0.01, 0.1, 0.5]
    alpha_results = {}
    
    for alpha in alphas:
        print(f"Testing α = {alpha}...")
        Q_alpha, _ = constant_alpha_monte_carlo_control(
            env, num_episodes=20000, alpha=alpha, epsilon=0.1
        )
        policy_alpha = extract_policy(Q_alpha)
        
        def alpha_policy(state):
            return policy_function(state, policy_alpha)
        
        results = test_policy(env, alpha_policy, 3000)
        alpha_results[alpha] = results['avg_reward']
    
    print("\nLearning Rate Results:")
    for alpha, reward in alpha_results.items():
        print(f"α = {alpha:5.3f}: Average Reward = {reward:.4f}")
    
    best_alpha = max(alpha_results, key=alpha_results.get)
    print(f"\nBest learning rate: α = {best_alpha}")
    
    print("\nExercise 3: Exploration Impact")
    print("-" * 30)
    
    epsilons = [0.01, 0.05, 0.1, 0.2, 0.3]
    epsilon_results = {}
    
    for epsilon in epsilons:
        print(f"Testing ε = {epsilon}...")
        Q_epsilon, _ = constant_alpha_monte_carlo_control(
            env, num_episodes=20000, alpha=0.01, epsilon=epsilon
        )
        policy_epsilon = extract_policy(Q_epsilon)
        
        def epsilon_policy(state):
            return policy_function(state, policy_epsilon)
        
        results = test_policy(env, epsilon_policy, 3000)
        epsilon_results[epsilon] = results['avg_reward']
    
    print("\nExploration Rate Results:")
    for epsilon, reward in epsilon_results.items():
        print(f"ε = {epsilon:5.3f}: Average Reward = {reward:.4f}")
    
    best_epsilon = max(epsilon_results, key=epsilon_results.get)
    print(f"\nBest exploration rate: ε = {best_epsilon}")
    
    print("\n" + "="*60)
    print("EXERCISES COMPLETE")
    print("="*60)
    print("\nKey Observations:")
    print("1. Policy design significantly impacts performance")
    print("2. Learning rate affects convergence speed and stability")
    print("3. Exploration rate balances learning vs performance")
    print("4. Hyperparameter tuning is crucial for optimal results")

# Run interactive exercises
interactive_exercises()

In [ ]:
def monte_carlo_tree_search_preview():
    """
    Preview of Monte Carlo Tree Search (MCTS) concepts
    """
    print("\n" + "="*60)
    print("BONUS: MONTE CARLO TREE SEARCH PREVIEW")
    print("="*60)
    
    print("\nMonte Carlo Tree Search (MCTS) extends MC methods:")
    print("\n1. **Selection**: Navigate tree using UCB1")
    print("2. **Expansion**: Add new nodes to tree")
    print("3. **Simulation**: Run random rollout")
    print("4. **Backpropagation**: Update all nodes in path")
    
    print("\nKey Differences from Standard MC:")
    print("- **Tree Structure**: Builds search tree incrementally")
    print("- **UCB1 Selection**: Balances exploration/exploitation")
    print("- **Partial Episodes**: Can work with incomplete information")
    print("- **Planning**: Looks ahead rather than just learning")
    
    print("\nApplications:")
    print("- Game AI (Go, Chess, Poker)")
    print("- Resource allocation")
    print("- Combinatorial optimization")
    print("- Real-time decision making")
    
    print("\nSimple MCTS-inspired action selection:")
    
    def ucb1_action_selection(Q, N, state, c=1.414):
        """
        UCB1 action selection (simplified)
        """
        if state not in N or sum(N[state]) == 0:
            return random.choice([0, 1])
        
        total_visits = sum(N[state])
        ucb_values = []
        
        for a in range(2):
            if N[state][a] == 0:
                ucb_values.append(float('inf'))
            else:
                exploitation = Q[state][a]
                exploration = c * np.sqrt(np.log(total_visits) / N[state][a])
                ucb_values.append(exploitation + exploration)
        
        return np.argmax(ucb_values)
    
    # Demonstrate UCB1 vs ε-greedy
    Q_demo = {(20, 10, False): np.array([0.5, -0.2])}
    N_demo = {(20, 10, False): np.array([100, 10])}
    
    state_demo = (20, 10, False)
    
    print(f"\nExample state: {state_demo}")
    print(f"Q-values: STICK={Q_demo[state_demo][0]:.2f}, HIT={Q_demo[state_demo][1]:.2f}")
    print(f"Visit counts: STICK={N_demo[state_demo][0]}, HIT={N_demo[state_demo][1]}")
    
    ucb1_action = ucb1_action_selection(Q_demo, N_demo, state_demo)
    greedy_action = np.argmax(Q_demo[state_demo])
    
    print(f"\nGreedy choice: {'STICK' if greedy_action == 0 else 'HIT'}")
    print(f"UCB1 choice: {'STICK' if ucb1_action == 0 else 'HIT'}")
    print("\nUCB1 considers both value and uncertainty!")

# Run MCTS preview
monte_carlo_tree_search_preview()

In [ ]:
# Final summary of learned policies
print("\n" + "="*80)
print("FINAL SUMMARY: MONTE CARLO METHODS IN BLACKJACK")
print("="*80)

print("\n🎯 MISSION ACCOMPLISHED:")
print("✅ Implemented Monte Carlo Evaluation")
print("✅ Implemented Monte Carlo Control")
print("✅ Explained Q-value vs V-value estimation")
print("✅ Demonstrated MDP to MRP transformation")
print("✅ Applied constant-α learning")
print("✅ Found optimal policy without analysis")
print("✅ Explored advanced MC techniques")
print("✅ Conducted interactive experiments")

print("\n📊 PERFORMANCE SUMMARY:")
print(f"Simple Policy Performance:     {simple_results['avg_reward']:.4f}")
print(f"Monte Carlo Optimal Policy:    {optimal_results['avg_reward']:.4f}")
print(f"Improvement:                   {optimal_results['avg_reward'] - simple_results['avg_reward']:.4f}")
print(f"Win Rate Improvement:          {optimal_results['win_rate'] - simple_results['win_rate']:.4f}")

print("\n🧠 KEY LEARNINGS:")
print("1. Monte Carlo methods learn from complete episodes")
print("2. Q-values enable model-free policy improvement")
print("3. Constant-α provides adaptive learning")
print("4. Exploration is crucial for finding optimal policies")
print("5. Different MC variants have different trade-offs")

print("\n🚀 NEXT STEPS:")
print("- Temporal Difference Learning (TD)")
print("- Q-Learning and SARSA")
print("- Deep Q-Networks (DQN)")
print("- Policy Gradient Methods")
print("- Actor-Critic Methods")

# Save final policy for future use
import pickle

final_results = {
    'optimal_policy': optimal_policy_dict,
    'Q_values': Q_optimal,
    'performance': optimal_results,
    'training_rewards': rewards
}

# Uncomment to save results
# with open('monte_carlo_blackjack_results.pkl', 'wb') as f:
#     pickle.dump(final_results, f)
# print("\n💾 Results saved to 'monte_carlo_blackjack_results.pkl'")

print("\n🎉 MONTE CARLO LEARNING COMPLETE!")
print("Thank you for exploring Monte Carlo methods in Reinforcement Learning!")

# Close environment
env.close()

## Additional Resources and References

### 📚 Recommended Reading:

1. **Sutton & Barto (2018)**: "Reinforcement Learning: An Introduction" - Chapter 5
2. **Bertsekas (2019)**: "Reinforcement Learning and Optimal Control"
3. **Szepesvári (2010)**: "Algorithms for Reinforcement Learning"

### 🔗 Online Resources:

- [OpenAI Gymnasium Documentation](https://gymnasium.farama.org/)
- [Reinforcement Learning Course by David Silver](https://www.davidsilver.uk/teaching/)
- [CS285 Deep RL Course (UC Berkeley)](http://rail.eecs.berkeley.edu/deeprlcourse/)

### 🛠️ Implementation Notes:

#### Monte Carlo Evaluation:
```python
# Pseudocode
for episode in episodes:
    G = 0
    for t in reversed(episode):
        G = γ * G + R[t+1]
        if S[t] not in visited:
            Returns[S[t]].append(G)
            V[S[t]] = average(Returns[S[t]])
```

#### Monte Carlo Control:
```python
# Pseudocode
for episode in episodes:
    # Generate episode using ε-greedy policy
    G = 0
    for t in reversed(episode):
        G = γ * G + R[t+1]
        if (S[t], A[t]) not in visited:
            Q[S[t]][A[t]] += α * (G - Q[S[t]][A[t]])
            π[S[t]] = argmax_a Q[S[t]][a]
```

### 🎯 Exercise Solutions:

Try implementing these variations:

1. **Weighted Importance Sampling**: For off-policy learning
2. **Incremental Implementation**: Update estimates incrementally
3. **Discounting**: Experiment with γ < 1
4. **Different Exploration**: Try UCB or Boltzmann exploration

### 🐛 Common Pitfalls:

- **Insufficient Exploration**: Policy may converge to suboptimal solution
- **High Learning Rate**: Can cause instability
- **Low Learning Rate**: Slow convergence
- **Forgetting First-Visit**: Every-visit can introduce bias

### 🔬 Extensions to Explore:

1. **Monte Carlo Tree Search (MCTS)**
2. **Gradient Monte Carlo**
3. **Natural Policy Gradients**
4. **Monte Carlo with Function Approximation**